In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../src'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/home/enis/projects/nna/src


In [2]:
from datetime import timedelta
import datetime 
import os

from fileUtils import read_file_properties_v2,find_filesv2,list_files,get_labeled_exif
from fileUtils import get_audio,str2timestamp,query_audio
# from labeling_utils import splitmp3

# import pandas as pd
from pathlib import Path

# from sklearn.neighbors import KDTree
import numpy as np

# from pydub import AudioSegment


In [4]:
from annoy import AnnoyIndex


In [3]:
# list_files("/scratch/enis/data/nna/real_2organize/")

In [4]:
# from sklearn.metrics.pairwise import paired_distances
# from sklearn.metrics import pairwise_distances
# from annoy import AnnoyIndex


# def paired_distances_broadcast(X,Y):
#     return paired_distances(np.repeat(X,Y.shape[0],axis=0), Y)



# def sigmoid(X):
#     return 1/(1+np.exp(-X))

# def sigmoidT(X,t):
#     return 1/(1+np.exp(-X*t))

from fileUtils import save_to_csv
# import csv
# def save_to_csv(file_name,lines):
#     file_name=Path(file_name).with_suffix('.csv')
#     with open(file_name, mode='a') as labels_file:
#         label_writer = csv.writer(labels_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#         for line in lines:
#             label_writer.writerow(line)

In [5]:
# output = find_filesv2(location,start_time,end_time,length,0,file_properties_df)
# sorted_filtered,start_time,end_time,start_time_org,end_time_org = output

#### find the embedding for a given image

In [6]:
# p_path="/home/enis/projects/nna/data/cameraTraps/test.txt"

# with open(p_path) as p_file:
#     photo_paths=p_file.readlines()
#     photo_paths = [i.strip() for i in photo_paths]

# other_folder=["anwr_35_2019_101RECNX_RCNX3373.JPG",
# "anwr_42_2019_100RECNX_RCNX3574.JPG",
# "ivvavik_SINP03_2019_100RECNX_IMG_3219.JPG",
# "ivvavik_SINP05_2019_100RECNX_IMG_2867.JPG",
# "ivvavik_SINP06_2019_100RECNX_IMG_1884.JPG",
# "ivvavik_SINP09_2019_100RECNX_IMG_2743.JPG",
# "prudhoe_17_2019_100RECNX_RCNX3916.JPG",]

# parent_path1="/tank/data/nna/examples/randomPhotos10k/"
# parent_path2="/tank/data/nna/examples/randomPhotos1k/"

# # photo with Caribou
# photo_paths=['anwr_37_2019_100RECNX_RCNX9317.jpg']

# given image paths finds files, code is at:
# notebooks/explore/get_audio4photos.ipynb

In [7]:
file_properties_df=pd.read_pickle("../../data/realdata_v2No_stinchcomb.pkl")

### 1. pick a sound to use for queries

In [9]:
# sound with Caribou

!find /scratch/enis/data/nna/real/ -iname "S4A10279_20190605_091602*"

/scratch/enis/data/nna/real/anwr/37/2019/S4A10279_20190605_091602_FCmodel
/scratch/enis/data/nna/real/anwr/37/2019/S4A10279_20190605_091602_FCmodel/S4A10279_20190605_091602_FCmodel000.npy
/scratch/enis/data/nna/real/anwr/37/2019/S4A10279_20190605_091602_vgg
/scratch/enis/data/nna/real/anwr/37/2019/S4A10279_20190605_091602_vgg/S4A10279_20190605_091602_rawembeddings000.npy
/scratch/enis/data/nna/real/anwr/37/2019/S4A10279_20190605_091602_vgg/S4A10279_20190605_091602_embeddings000.npy


In [69]:
metadata[0]

('/scratch/enis/data/nna/real/anwr/31/2019/S4A10297_20190504_000000_vgg/S4A10297_20190504_000000_embeddings000.npy',
 4560)

In [4]:
# aircraft sound

!find /scratch/enis/data/nna/real/ -iname "S4A10255_20190507_073000*"


^C


### 1.1 find similar embeddings for given embeddings in the same location 

In [67]:
# sound with Caribou

raw_embed_name="/scratch/enis/data/nna/real/anwr/37/2019/S4A10279_20190605_091602_vgg/S4A10279_20190605_091602_rawembeddings000.npy"
raw_embed=np.load(raw_embed_name)


In [8]:
# aircraft sound
# raw_embed_name="/scratch/enis/data/nna/real/prudhoe/26/2019/S4A10255_20190507_073000_vgg/S4A10255_20190507_073000_rawembeddings000.npy"
raw_embed_name="/scratch/enis/data/nna/real/prudhoe/26/2019/S4A10255_20190507_073000_vgg/S4A10255_20190507_073000_embeddings000.npy"

raw_embed=np.load(raw_embed_name)
raw_embed=raw_embed.astype(np.float32)


In [240]:
# S4A10255_20190507_073000-1808second
(30*60)+8

1808

### 1.2 Slice embedding for exact time of interest from a big file

In [10]:
# aircraft sound
start_seconds=(30*60)+8
raw_embed_audio=raw_embed[int(start_seconds):int(start_seconds)+60]

In [9]:
# sound with Caribou
start_seconds=(start_time-sorted_filtered["timestamp"])[0].total_seconds()
raw_embed_audio=raw_embed[int(start_seconds):int(start_seconds)+60]

NameError: name 'start_time' is not defined

In [12]:
raw_embed_audio.shape

(60, 128)

In [2]:
from nna.fileUtils import list_files

### 2.1 load embeddings to search from 

In [5]:
search_path="/scratch/enis/data/nna/real/"
filenamePattern="*_embeddings*.npy" #raw would be "*_rawembeddings*.npy"
fn='/scratch/enis/data/nna/realMerged/allEmbeddingsV11.ann'


In [30]:
# !rm /scratch/enis/data/nna/realMerged/allEmbeddingsV11.ann

In [6]:
%%time
# find all files
import glob

all_embedding_paths=list_files(search_path,filename=filenamePattern)


CPU times: user 1min 8s, sys: 2min 5s, total: 3min 13s
Wall time: 16min 13s


In [17]:
# filtered=[e for e in all_embeddings if "_embeddings" in e]


'/scratch/enis/data/nna/real/anwr/31/2019/S4A10297_20190505_040000_vgg/S4A10297_20190505_040000_embeddings000.npy'

In [28]:
all_embedding_paths=all_embeddings

In [16]:
%%time
# load and merge embedding files
#LOAD:
embeds=[None]*len(all_embeddings)
index=0
for i,embed_file in enumerate(all_embeddings):
    embed=np.load(embed_file)
    index+=embed.shape[0]
    embeds[i]=(embed)
#MERGE:
# Faster then np.concatenate
concat_embeds=np.zeros([index,128],dtype=np.float32)
index=0
for i,embed in enumerate(embeds):
    concat_embeds[index:(index+embed.shape[0]),:]=embed[:]
    index+=embed.shape[0]

CPU times: user 956 ms, sys: 816 ms, total: 1.77 s
Wall time: 22.5 s


In [32]:
!conda run -n speechEnv python /home/enis/projects/nna/src/slack_message.py -m "annoy ready "


In [57]:
embedSizes=[]
for i,embed_file in enumerate(all_embedding_paths):
    embed=np.load(embed_file)
    embedSizes.append(embed.shape[0])
    if i%1000==0:
        print(i/len(all_embedding_paths))

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [59]:
metadataCSV="/scratch/enis/data/nna/realMerged/"+"allEmbeddings_"+"V11"+".csv"

metadata=list(zip(all_embeddings,embedSizes))
save_to_csv(metadataCSV,metadata)

### create an ANNOY index for files at all_path on disk

In [31]:
%%time
import time
# vector size
f=128
t = AnnoyIndex(f, 'euclidean')  # Length of item vector that will be indexed
# for i in range(concat_embeds.shape[0]):
#     t.add_item(i, concat_embeds[i,:])
#prepares annoy to build the index in the specified file instead of RAM 
#(execute before adding items, no need to save after build)
t.on_disk_build(fn)
index=0
start=time.time()
for i,embed_file in enumerate(all_embedding_paths):
    embed=np.load(embed_file)
    for ii in range(embed.shape[0]):
        t.add_item(index, embed[ii,:])
        index+=1
    if i%1000==0:
        end=time.time()
        print(i//1000,end-start)
        start=end


0 0.8027288913726807
0 31.712205171585083
0 31.272992849349976
0 27.54176688194275
0 31.563138008117676
0 23.924992084503174
0 27.800413131713867
0 30.466721296310425
0 26.936872482299805
0 29.223549127578735
1 24.20743489265442
1 28.79265570640564
1 28.168747425079346
1 22.139662981033325
1 39.686907052993774
1 26.57844591140747
1 24.71869134902954
1 26.688180446624756
1 28.306283950805664
1 29.85997772216797
2 30.870782613754272
2 30.365631103515625
2 29.58386468887329
2 28.879225492477417
2 29.51815176010132
2 31.011958360671997
2 25.854575634002686
2 27.967524528503418
2 24.453100204467773
2 28.058457136154175
3 27.45342493057251
3 26.040910243988037
3 29.451515436172485
3 27.987468719482422
3 27.627418279647827
3 25.75637912750244
3 28.31390643119812
3 28.2982656955719
3 27.579447507858276
3 27.807769536972046
4 27.339429140090942
4 27.705240964889526
4 29.781339168548584
4 22.505403757095337
4 27.928438901901245
4 28.74213695526123
4 28.02622890472412
4 28.17542028427124
4 27.216

In [ ]:
try:
    t.set_seed(42)
    t.build(24)
    !conda run -n speechEnv python /home/enis/projects/nna/src/slack_message.py -m "big file ready "
except Exception as ex:
    print(ex)
    !conda run -n speechEnv python /home/enis/projects/nna/src/slack_message.py -m "big file failed"


In [54]:
print("test")

test


In [27]:
embed.shape,embed_file

((456,),
 '/scratch/enis/data/nna/real/anwr/31/2019/S4A10297_20190504_000000_AIRCRAFT/S4A10297_20190504_000000_AIRCRAFT000.npy')

In [65]:
t.get_distance(100, 0)

336.4030456542969

In [22]:
# t.build(10)

True

In [64]:
t.get_nns_by_item(10, 12, include_distances=True)

([10,
  52461816,
  96609140,
  76790561,
  88836370,
  101769213,
  91824888,
  107618027,
  87824292,
  20418699,
  30865637,
  77014901],
 [0.0,
  131.22499084472656,
  141.59449768066406,
  143.01048278808594,
  145.70518493652344,
  152.381103515625,
  152.59095764160156,
  156.58224487304688,
  157.10824584960938,
  157.56268310546875,
  158.90878295898438,
  159.6684112548828])

In [60]:
t.get_n_items()

185328590

In [50]:
185328590/(2**24)

11.046444773674011

In [40]:
(2**16)

65536

In [66]:
# t.get_n_items()
len(all_path),len(all_embeddings)

NameError: name 'all_path' is not defined

In [13]:
!ls -alh /scratch/enis/data/nna/realMerged/allEmbeddings.ann

total 102G
drwxrwxr-x 2 enis enis    5 Apr  9 12:44 .
drwxrwxr-x 9 enis enis    9 Apr 21 12:47 ..
-rw------- 1 enis enis 132G Apr  9 21:59 allEmbeddings.ann
-rw------- 1 enis enis 6.2K Apr  9 12:49 prudhoe26_1.ann
-rw------- 1 enis enis 1.1G Apr  9 12:42 prudhoe26.ann


In [13]:
# concat_embeds.shape

(1732054, 128)

In [14]:
# make sure it multiples of 10 to make things easy
concat_embeds=concat_embeds[:(concat_embeds.shape[0]//10)*10,:]

### there are three methods for creating vectors:

In [15]:
# # Method 1   1 second 
excerptLen=1
exp_name="1second"
concat_embeds2=concat_embeds[:]
raw_embed_audio2=raw_embed_audio[:]
raw_embed_audio2=raw_embed_audio2[20:21,:]
rowN=100000

#Method 2    mean 10 second
# excerptLen=10
# exp_name="Mean10Second"
# concat_embeds2=concat_embeds.reshape(-1,10,128).mean(axis=1)
# raw_embed_audio2=raw_embed_audio.reshape(-1,10,128).mean(axis=1)
# raw_embed_audio2=raw_embed_audio2[2:3,:]
# rowN=10000

# #Method 3    concat 10 second
# excerptLen=10
# exp_name="Concat10Second"
# concat_embeds2=concat_embeds.reshape(-1,1280)
# raw_embed_audio2=raw_embed_audio.reshape(-1,1280)
# raw_embed_audio2=raw_embed_audio2[2:3,:]
# rowN=10000



In [16]:
concat_embeds.shape

(1732050, 128)

In [17]:
# concat_embeds3=concat_embeds[:,:]

* we are trying to create a matrix, rows are queries and columns are data points in database
* values are similarity, 
* then 


In [18]:
raw_embed_audio.shape

(60, 128)

In [18]:

# paired_distances(raw_embed_audio, raw_embed_audio)

In [174]:
# %%time
# # t.build(10) # 10 trees

# # u = AnnoyIndex(f, 'angular')
# # u.load('test.ann') # super fast, will just mmap the file
# a=(t.get_nns_by_item(1001, 100)) # will find the 1000 nearest neighbors
# b=(t.get_nns_by_item(1000, 100))

# a=np.array(a)
# b=np.array(b)

# numbers=set()
# for i in range(10):
#     numbers=numbers.union(set(a).intersection(set(b+i)))
#     numbers=numbers.union(set(a).intersection(set(b-i)))
# len(numbers)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 738 µs


1

In [68]:
%%time
for m in range(10):
#     allA=[None]*concat_embeds.shape[0]
    for i in range(concat_embeds.shape[0]):
        a=u.get_distance(i, 0)

CPU times: user 8.36 s, sys: 52 ms, total: 8.41 s
Wall time: 8.42 s


In [69]:
%%time
import multiprocessing

def worker(index,numberItems):
    allA=[None]*numberItems

    for i in range(numberItems):
        a=u.get_distance(i, index)
    return None

jobs = []
for index in range(10):
    numberItems=u.get_n_items()
    p = multiprocessing.Process(target=worker,args=(index,numberItems))
    jobs.append(p)
    p.start()

for p in jobs:
    p.join()

CPU times: user 52 ms, sys: 788 ms, total: 840 ms
Wall time: 2.08 s


In [51]:
allA[4]

745.1812133789062

In [55]:
results[0]

array([1224.9686, 1126.4288, 1097.2521, ..., 1204.9872, 1104.8918,
       1191.6367], dtype=float32)

In [62]:
%%time
# one sample/query
# res=paired_distances_broadcast(raw_embed_audio2,concat_embeds2)
# res.shape

#
# 10 queries
queryCount=10
startSecond=0
excerptLen=1

gamma=1/512
startIndex=int(startSecond/excerptLen)
endIndex=startIndex+queryCount


results = pairwise_distances(raw_embed_audio[startIndex:10,:], concat_embeds)

# results=np.exp(-results*gamma)




CPU times: user 11.3 s, sys: 18.1 s, total: 29.5 s
Wall time: 3.78 s


In [53]:
%%time
results=np.exp(-results*gamma)

CPU times: user 16 ms, sys: 3.27 s, total: 3.29 s
Wall time: 3.29 s


In [32]:
# I might have to use this method, if I want to get maximum without replacement

# %%time
# stepSize=1
# windowSize=10
# resultsMax=[]
# resultsAvg=[]

# windowMax=np.max(results2,axis=0)

# for i in range(0,results2.shape[1],stepSize):
# #     windowElements=(results2[:,i:(i+windowSize+1)])
# #     colMax=np.max(windowElements,axis=0)

#     resultAvg=np.average(colMax)
#     resultsAvg.append(resultAvg)
# #     resultsMax.append(windowMax)


CPU times: user 29.9 s, sys: 960 ms, total: 30.8 s
Wall time: 29.8 s


### Run sliding window over the Matrix

In [491]:
%%time
windowSize=10
windowMax=np.max(results,axis=0)
windowMean=pd.Series(windowMax).rolling(window=windowSize).mean().iloc[windowSize-1:].values


CPU times: user 2.31 s, sys: 1min 3s, total: 1min 5s
Wall time: 1min 5s


In [ ]:
%%time
windowSize=10
windowMax=np.max(resultMatrix,axis=0)
windowMean=pd.Series(windowMax).rolling(window=windowSize).mean().iloc[windowSize-1:].values


### Look into stats of results

In [203]:
from scipy import stats

stats.describe(res)

DescribeResult(nobs=1732041, minmax=(0.05875636413693428, 0.3321208477020264), mean=0.12500929506519953, variance=0.00040337682893698085, skewness=1.7658461475250447, kurtosis=8.105885715994328)

In [201]:
res=windowMean[:]

### find kth highest scores (similarity of queries)

In [334]:
# arr = np.array([1, 3, 2, 4, 5,6,7,8,8,1,1,9])
kth=250
ind = np.argpartition(res, -kth)[-kth:]
sortedbyScore=sorted(list(zip(res[ind],ind)),reverse=True)

#### Highest scores might be consequtive, merging these as one result helps decrease amount of files to listen

In [12]:
# find consequtive points that has high value 

theta=5
sortedbyInd=sorted(list(zip(res[ind],ind)),key=lambda x:x[1])
previous=sortedbyInd[0][1]
count=0
series=[]
starts=[]
for score,index in sortedbyInd:
    if index<=previous+theta:
        series.append((score,index))
    else:
        starts.append(series)
        series=[]
        series.append((score,index))
    previous=index

# sorted(starts,key=lambda x:x[1],reverse=True)
print("total periods",len(starts))
previous=0
print("short distances")
for score_indexes in starts:
    if (score_indexes[-1][1]-score_indexes[0][1])>30:
        print(score_indexes[0][1],score_indexes[-1][1])


NameError: name 'res' is not defined

### generating figures for similarity scores

In [152]:
res.shape,rowN,res.shape[0]/rowN

((1732041,), 100000, 17.32041)

In [153]:
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker

minY,maxY=min(res),max(res)
exp_name="slidingWindow_AverageOfMax_short20k"
name=f"Euclidian Similarity {exp_name}"
nrows=res.shape[0]//rowN
print(nrows)
fig, ax = plt.subplots(nrows=nrows,figsize=(200, 20*nrows))
for i,axes in enumerate(ax):
    axes.plot(res[i*(rowN):(i+1)*(rowN)])
    axes.set_ylim(minY,maxY)
    axes.grid(True)

ax[0].set_ylabel(f"{name}",fontsize=32)

# plt.grid(True)

# loc = plticker.MultipleLocator(base=5000) # this locator puts ticks at regular intervals
# ax.xaxis.set_major_locator(loc)

plt.tight_layout()
fig.show()
filename=name.replace(" ","_")
fig.savefig(f"/home/enis/projects/nna/results/vis/nearestNeighbour/{filename}.png")


17


In [169]:
excerptLen=10
exp_name="Concat10Second"
concat_embeds2=concat_embeds.reshape(-1,1280)
raw_embed_audio2=raw_embed_audio.reshape(-1,1280)
raw_embed_audio2=raw_embed_audio2[2:3,:]
rowN=10000

res=paired_distances_broadcast(raw_embed_audio2,concat_embeds2)
res.shape

import matplotlib.pyplot as plt
import matplotlib.ticker as plticker

name=f"Euclidian Distance {exp_name}"
nrows=res.shape[0]//rowN

fig, ax = plt.subplots(nrows=nrows,figsize=(200, 20*nrows))
for i,axes in enumerate(ax):
    axes.plot(res[i*(rowN):(i+1)*(rowN)])

ax[0].set_ylabel(f"{name}",fontsize=32)

plt.grid(True)

# loc = plticker.MultipleLocator(base=5000) # this locator puts ticks at regular intervals
# ax.xaxis.set_major_locator(loc)

plt.tight_layout()
# fig.show()
filename=name.replace(" ","_")
fig.savefig(f"/home/enis/projects/nna/results/vis/nearestNeighbour/{filename}.png")


In [232]:
len(all_embeddings),excerptLen

(385, 10)

In [310]:
# sorted([sorted(start) for start in starts])

In [ ]:
split_folder="/home/enis/projects/nna/data/nearestNeighbours/S4A10255_20190507_073000-1808seconds/slidingWindow_AverageOfMax/"
excerptLen=1
# for indexes in (starts):
#     score,index=(indexes[len(indexes)//2])
#     print(score)
#     print(embedIndex2fileSecond(index,embeds,all_embeddings,excerptLen=1))
    
for score,index in (sortedbyScore):
#     score,index=(indexes[len(indexes)//2])
    print(score)
    print(embedIndex2fileSecond(index,embeds,all_embeddings,excerptLen=1))

# indexes=[indexes[len(indexes)//2] for indexes,count in (starts)]
# # # indexes
# clipAddBip(indexes,exp_name,split_folder,embeds,all_embeddings,excerptLen=10,reductionLen=1)
    


In [50]:
kdt = KDTree(concat_embeds2, leaf_size=30, metric='euclidean')
distances,indexes=kdt.query(raw_embed_audio2, k=10, return_distance=True)
allResults=[ (distance,index) for index,distance in (zip(indexes.flatten(),distances.flatten()))]
allResults.sort()
# distances,indexes=allResults

In [51]:
# distances

In [8]:
def embedIndex2fileSecond(index,embeds,all_embeddings,excerptLen=1):
    alist=[]
    countStart=0
    embed_count=index*excerptLen
    for i,em in enumerate(embeds):
        countEnd=countStart+(em.shape[0])
        if countStart<=embed_count and embed_count<=countEnd:
            startSecond=embed_count-countStart
    #             print(i,countStart,countEnd,"start second:",startSecond)
            filename=("/tank/"+"/".join(all_embeddings[i].split("/")[3:-2])+"/"+all_embeddings[i].split("/")[-2][:-4]+".flac")
#             alist.append((filename,startSecond))
            return((filename,startSecond))
#             break
        countStart=countEnd

    

In [9]:
split_folder="/home/enis/projects/nna/data/nearestNeighbours/"
def clipAddBip(allIndexes,exp_name,split_folder,embeds,all_embeddings,excerptLen,reductionLen):
    # embed_count=823804
    beep_wav="/home/enis/projects/nna/data/beep.wav"
    beep_wav = AudioSegment.from_file(beep_wav)
    beep_wav=beep_wav-30

    split_folder=(split_folder+exp_name)
    Path(split_folder).mkdir(parents=True, exist_ok=True)

    for index in allIndexes[:]:
        (filename,startSecond)=embedIndex2fileSecond(index,embeds,all_embeddings,excerptLen=reductionLen)
        endSecond=startSecond+excerptLen
        print(filename,startSecond)
        input_file=filename
        buffer=10
        start_time="{}.{}".format((startSecond-buffer)//60,(startSecond-buffer)%60)
        end_time="{}.{}".format((endSecond+buffer)//60,(endSecond+buffer)%60)
        outputSuffix=".wav"
        output_file = splitmp3(input_file,split_folder,start_time,end_time,depth=1,backend="ffmpeg",outputSuffix=outputSuffix)

        other_wav=AudioSegment.from_file(output_file)
        start,end=(buffer*1000),((buffer+excerptLen)*1000)
        result_wav=other_wav[:start]+(beep_wav)+other_wav[start:end]+beep_wav+other_wav[end:]
        o_file=result_wav.export(output_file)



In [37]:
# o_file=result_wav.export(output_file)

In [8]:
# len(other_wav)

(60000, 2)

In [5]:
# beep_wav="/home/enis/projects/nna/data/beep.wav"
# other_wav="./S4A10279_20190507_033000_4008-4068.wav"
# beep_wav = AudioSegment.from_file(beep_wav)
# beep_wav=beep_wav-30
# other_wav=AudioSegment.from_file(other_wav)

In [ ]:
# other_wav[:start]+(beep_wav-30)+other_wav[start:end]+

In [43]:
# res=(beep_wav-30)+other_wav[:1000]


<_io.BufferedRandom name='mashup.wav'>

In [4]:
# input_file='/tank/data/nna/real/anwr/37/2019/S4A10279_20190605_091602.flac'
# split_folder="./"
# start_time="2.12"
# end_time="3.12"
# outputSuffix=".wav"
# a=splitmp3(input_file,split_folder,start_time,end_time,depth=1,backend="ffmpeg",outputSuffix=outputSuffix)


S4A10279_20190605_091602_2m_12s__3m_12s.wav


In [ ]:
# !ffmpeg -i '/tank/data/nna/real/anwr/37/2019/S4A10279_20190605_091602.flac' -ss 304 -to 364 -c copy S4A10279_20190605_091602_304-364.wav


# !ffmpeg -i '/tank/data/nna/real/anwr/37/2019/S4A10279_20190604_234602.flac' -ss 650 -to 700 -c copy S4A10279_20190604_234602_650-700.wav


# !ffmpeg -i '/tank/data/nna/real/anwr/37/2019/S4A10279_20190509_073000.flac' -ss 4149 -to 4209 -c copy S4A10279_20190509_073000_4149-4209.wav

# !ffmpeg -i '/tank/data/nna/real/anwr/37/2019/S4A10279_20190530_214602.flac' -ss 3260 -to 3320 -c copy S4A10279_20190530_214602_3260-3320.wav

# !ffmpeg -i '/tank/data/nna/real/anwr/37/2019/S4A10279_20190507_033000.flac' -ss 4008 -to 4068 -c copy S4A10279_20190507_033000_4008-4068.wav

# !ffmpeg -i '/tank/data/nna/real/anwr/37/2019/S4A10279_20190507_033000.flac' -ss 320 -to 380 -c copy segment8.wav

# !ffmpeg -i '/tank/data/nna/real/anwr/37/2019/S4A10279_20190608_020000.flac' -ss 222 -to 282 -c copy S4A10279_20190608_020000_222-282.wav




In [ ]:
# !mp3splt -d "./" -f -D /tank/data/nna/real/anwr/37/2019/S4A10279_20190517_130000.flac 15.27 16.27



In [ ]:
# ffmpeg -i '/tank/data/nna/real/anwr/37/2019/S4A10279_20190605_091602.flac' -ss 304 -to 364 -c copy S4A10279_20190605_091602_304-364.wav



In [68]:
# !ls/tank/data/nna/real/anwr/37/2019/S4A10279_20190603_194602.flac

books_read.png			get_audio4photos.ipynb
cum_hist_1000bins_025limit.svg	listen_photos.ipynb
cum_hist_1000bins.png		listen_segments_backup.ipynb
cum_hist_1000bins.svg		listen_segments.ipynb
cumulative_histogram2.svg	nearest_neighbors.ipynb
cumulative_histogram.jpg	Untitled1.ipynb
cumulative_histogram.png	Untitled.ipynb
